In [1]:
# Standard Library
import os
import glob
import copy
import random
import argparse

# Third-Party Libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, Subset

# Local Modules
import architecture
import trainer
from utils import *

In [2]:
# Data information
num_channels = 19   # number of channels
num_classes  = 2    # number of classes
samp_freq    = 500  # Sampling frequency

# Architecture 
MP           = [10.0, 2.0]     # Multi-resolution list in seconds
feat_dim     = 32              # Feature dimension

# Training parameters
batch_size   = 32              # Number of images in each mini-batch
lr_rate      = 1e-03           # Learning-rate
patience     = 30              # patience 
tr_val_ratio = 0.7             # Train and Validation ratio 
class_ratio  = 2.0             # seizure nonseizure class ratio
num_epochs   = 10              # Number of sweeps over the dataset to train


num_timepoints = int(MP[0]*samp_freq)

# Train data ----------------------
num_train_sz_samples = 100
num_train_ns_samples = 1000
train_sz   = np.random.randn(num_train_sz_samples, num_channels, num_timepoints).astype(np.float32) # Random EEG-like data 
train_ns   = np.random.randn(num_train_ns_samples, num_channels, num_timepoints).astype(np.float32) # Random EEG-like data


# Test data ----------------------
num_test_samples = 30
test_data  = np.random.randn(num_test_samples, num_channels, num_timepoints).astype(np.float32) # Random EEG-like data
test_label = np.random.randint(0, 2, size=(num_test_samples,)).astype(np.int64) # Random integer labels (e.g., 0 or 1 for binary classification)

In [3]:
# Definition of Model
total_feat_len = int(np.sum([feat_dim * (MP[0] // s) for s in MP]))
print(f"total_feat_len: {total_feat_len}")

Model_mr = architecture.Model(  n_chans=num_channels, 
                                n_classes=num_classes, 
                                feature_dim=feat_dim, 
                                Fs=samp_freq, 
                                SEC=MP,
                                tot_feat_len=total_feat_len).float()

total_feat_len: 192


In [4]:
save_dir = "results"
os.makedirs(save_dir, exist_ok=True)

model_dir = os.path.join(save_dir, "model.pt")
loss_dir  = os.path.join(save_dir, "loss.csv")


model_trainer = trainer.Trainer(Model_mr, class_ratio, num_classes)
model_trainer.compile(learning_rate=lr_rate)
Tracker = model_trainer.train(  train_sz=train_sz, 
                                train_ns=train_ns, 
                                epochs=num_epochs, 
                                batch_size=batch_size, 
                                patience=patience,
                                tr_val_ratio = tr_val_ratio,
                                directory=model_dir, 
                                loss_dir=loss_dir)

test_dataset = Dataset_tensor(test_data,  test_label)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

pred, feat, prob, tgt = model_trainer.predict(test_loader)


print("Shape of pred:", pred.shape)
print("Shape of feat:", feat.shape)
print("Shape of prob:", prob.shape)
print("Shape of tgt:",  tgt.shape)

Class weights: [0.75 1.5 ]


Valid Epoch 1/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.33it/s]


Epoch 001 | Train Loss: 7064.2871 | Val Loss: 6962.9952


Valid Epoch 2/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.23it/s]


Epoch 002 | Train Loss: 6961.1008 | Val Loss: 6929.1919


...

Epoch 009 | Train Loss: 6670.7604 | Val Loss: 6489.2790


Valid Epoch 10/10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.57it/s]


Epoch 010 | Train Loss: 6323.8899 | Val Loss: 6192.9564
# EDF Predict ------------------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.33it/s]

Shape of pred: (30,)
Shape of feat: (30, 192)
Shape of prob: (30, 2)
Shape of tgt: (30,)


In [5]:
accuracy, precision, recall, specificity, f1, macro_f1, roc_auc = predict2perf(pred, tgt, prob)

# Print results
print(f"Accuracy          : {accuracy:.4f}")
print(f"Precision         : {precision:.4f}")
print(f"Recall            : {recall:.4f}")
print(f"Specificity       : {specificity:.4f}")
print(f"F1 Score          : {f1:.4f}")
print(f"Macro F1          : {macro_f1:.4f}")
print(f"ROC-AUC           : {roc_auc:.4f}")

Accuracy          : 0.6000
Precision         : 0.0000
Recall            : 0.0000
Specificity       : 1.0000
F1 Score          : 0.0000
Macro F1          : 0.3750
ROC-AUC           : 0.5972
